## Task_18

##### Site:  https://erefdn.org

Utiliza o BeautifulSoup para processar a página.
Estrutura da página:
* Página principal: Possui uma lista com URLs para cada bolsa disponível
* Páginas dos Artigos: Uma página dedicada para cada bolsa, contendo todas as suas informações.




##### Processamento da página principal

O processamento da página principal percorre a lista de bolsas disponíveis, armazenando-as em uma lista.

<img src="imgs\principal.jpg" style="width: 500px;"/>

##### Processamento das páginas de cada bolsa (artigos)

Uma função encapsula todo o processamento de um artigo. Apenas a sua URL é passada.
O retorno ocorre através de um objeto JSON contendo os campos pertinentes.

<img src="imgs\artigo.jpg" style="width: 500px;"/>

### Imports

In [141]:
from bs4 import BeautifulSoup
import requests
import re
from lxml import html
from lxml import etree
import json
import pandas as pd
from time import sleep

### Definições Iniciais

In [142]:
url_pag_principal = 'https://erefdn.org/research-grants-projects/currently-funded-projects'

In [143]:
end_arq_csv_saida = './saida/resultado-raspagem-erefdn.csv'

In [144]:
tempo_espera_entre_chamadas =  0.25

In [145]:
id_site = 'erefdn'

### Funções Auxiliares

In [146]:
def obtem_soup(url):
    """Obtem o objeto Soup para uma URL
    Recebe:
            url :: str
    Retorna:
            soup :: bs4.BeautifulSoup
    """
    
    headers = {'User-Agent': 
               'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.3'}
    html = requests.get(url, headers=headers)
    soup = BeautifulSoup(html.text, 'lxml')
    
    return soup

In [147]:
def valor_int(str_valor):
    """Converte o valor financeiro da bolsa de string para float.
    Recebe:
            str_valor_usd :: str
    Retorna:
            valor_int :: int
    """
    chars_a_remover = ['$', 'U', 'S', ',', ' ']
    valor_inteiro = ''.join([char for char in str_valor if char not in chars_a_remover])
    return valor_inteiro

In [148]:
def remove_novas_linhas(str_bruta, sep='\n'):
    """Remove novas linhas '\n's em excesso e 
    devolve uma string com separador=sep.
    Recebe:
            str_bruta :: str
    Retorna:
            str_proc  :: str
            
    Exemplo:
        str_bruta = '\n\nCampo 1:\n\n\nValor 1\n\nValor2\n\n\n'
        str_proc = 'Campo 1:\nValor 1\nValor2'
    """
    return re.sub('\n+', sep, str_bruta).strip()

In [149]:
def gera_csv(lista_bolsas, end_arq_csv_saida):
    """Cria um arquivo de saida no formato CSV.
    Recebe:
            lista_bolsas :: list(dicts)
    Retorna:
            arquivo de saida  :: arquivo csv
            df_bolsas :: Pandas.DataFrame

    """
    print(f'Gerando o arquivo CSV de Saida em {end_arq_csv_saida}.')
    
    # Gera um dataframe a partir da lista de bolsas
    df_bolsas = pd.DataFrame(lista_bolsas)
    
    # Gera o CSV
    df_bolsas.to_csv(end_arq_csv_saida, sep=';', index=False)
    
    return df_bolsas

### Funções de Parsing

In [164]:
def parse_cabecalho(cabecalho):
    """Parse do cabecalho, Titulo e Investigators
    Recebe: 
            cabecalho do artigo :: soup
    Retorna:
            titulo :: str
            instituicao :: str
    """
    try:
        titulo = cabecalho.h1.text
        instituicao = cabecalho.p.text.split(':')[1].strip()
    except:
        print("!!! ERRO !!! - parse_cabecalho")
        titulo, instituicao = 'None', 'None'
    
    return titulo, instituicao

In [165]:
def parse_data_valor(data_valor):
    """Parse do Data de Inicio, Valor da bolsa
    Recebe: 
            data_valor :: soup
    Retorna:
            data_ini :: str
            valor :: str
    """
    
    tags_p = data_valor.findAll('p')
    
    try:
        data_ini = tags_p[0].text.split('Start Date:')[1].strip()
        valor = tags_p[1].text.split('Award Amount:')[1].strip()
    except:
        print("!!! ERRO !!! - parse_data_valor")
        data_ini, valor = 'None', 'None'
    
    return data_ini, valor

In [166]:
def parse_descricao(descricao):
    """Parse da descricao detalhada 
    Recebe: 
            descricao :: soup
    Retorna:
            descricao_detalhada :: str
    """
    try:
        descricao_detalhada = remove_novas_linhas(descricao.text)
    except:
        print("!!! ERRO !!! - parse_descricao")
        descricao_detalhada = 'None'
    
    return descricao_detalhada

In [153]:
def obtem_sessoes_artigo(soup_artigo):
    """Separa as partes de um artigo, conforme layout do site.
    
    Partes:
        Sessao 1 - Cabecalho, contendo (Titulo, Instituicao)
        Sessao 2 - Data e Valor, contendo (Data de Inicio, Valor concedido (USD))
        Sessao 3 - Descricao Detalhada
        
    Recebe: 
            soup_artigo :: soup
    Retorna:
            cabecalho :: soup
            data_valor :: soup
            descricao :: soup
    """    
    # Filtra apenas o conteudo de interesse
    conteudo_interesse_artigo = soup_artigo.find(class_="entry-content")
    elementos = conteudo_interesse_artigo.findAll(class_="vc_row wpb_row vc_row-fluid")
    
    # Divide o objeto soup em setores, conforme layout da pagina
    cabecalho = elementos[0]
    data_valor = elementos[1]
    descricao = elementos[2]
    
    return cabecalho, data_valor, descricao

In [154]:
def parse_lista_bolsas(soup_pag_princ):
    """Parse da pagina principal com 
       a lista de bolsas a ser coletada.
    Recebe: 
            soup da pagina
    Retorna:
            lista_bolsas:: list(dicts)
                titulo :: str
                url_artigo :: str
    """
    # Filtra a regiao de interesse dentro da pagina principal
    conteudo_interesse_pag_princ = soup_pag_princ.find(class_="entry-content")
    
    # Obtem a lista de bolsas disponiveis
    lista_bolsas = []
    tags_a = conteudo_interesse_pag_princ.findAll('a')

    # Para cada tag a, isola o link e o titulo do artigo e adiciona à lista de retorno
    for el in tags_bolsas:
        titulo = el.text
        url_artigo = el.get('href')

        lista_bolsas.append({'titulo': titulo, 'url_artigo': url_artigo})
  
    return lista_bolsas

### Funções Processamento Completo

In [155]:
def processa_artigo(url_artigo):
    """Processamento completo de um artigo (bolsa)
    Recebe: 
            url_artigo :: str
    Retorna:
            dict contendo:
                'titulo':: str, 
                'instituicao':: str,
                'data_ini':: str,
                'valor':: str,
                'descr_detalhada':: str
                'links':: str
    """
    # Obtem o objeto soup para uma bolsa especifica
    soup_artigo = obtem_soup(url_artigo)

    # Divide o objeto soup em setores, conforme layout da pagina
    cabecalho, data_valor, descricao = obtem_sessoes_artigo(soup_artigo)
    
    titulo, instituicao = parse_cabecalho(cabecalho)
    data_ini, valor = parse_data_valor(data_valor)
    descr_detalhada = parse_descricao(descricao)
        
    # Monta dicionario de retorno
    dados_bolsa = {'titulo': titulo, 
                   'instituicao': instituicao,
                   'data_ini': data_ini,
                   'valor': valor,
                   'descr_detalhada': descr_detalhada,
                   'links': url_artigo
                  }
    
    return dados_bolsa

In [156]:
def processa_pag_principal(url_pag_principal, log=False):
    """Processamento completo da pagina. 
    Gera a lista de bolsas disponiveis. Para cada bolsa, 
        faz o processamento completo dos dados.
    Recebe: 
            url do artigo :: str
    Retorna:
            lista_bolsas :: list(dicts) com o parsing completo de cada bolsa.
    """
    # Obtem o objeto soup para a pagina inicial, 
    # com links para todas as bolsas oferecidas
    soup_pag_princ = obtem_soup(url_pag_principal)
    
    # Obtem a lista de bolsas
    tit_url_bolsas = parse_lista_bolsas(soup_pag_princ)
    lista_bolsas = []
    
    # Para cada bolsa, processa o artigo correspondente, 
    # que fica em uma pag separada.
    for i, bolsa in enumerate(tit_url_bolsas):
        # Obtem informacoes basicas de cada bolsa
        titulo_artigo = bolsa['titulo']
        url_artigo = bolsa['url_artigo']
        print(f'---- Processando artigo {i + 1:2d}.. {titulo_artigo[:min(60, len(titulo_artigo))]}..')
        
        # Obtem o parsing completo da bolsa
        json_artigo = processa_artigo(url_artigo)
        
        # Salva o resultado na lista de jsons
        lista_bolsas.append(json_artigo)
        
        # Imprime resultado na tela se a flag de log for True
        if log == True:
            print(json.dumps(json_artigo, indent=4))
        
        sleep(tempo_espera_entre_chamadas)

    return lista_bolsas


### Execução

In [157]:
# Obtem a lista de parsings para cada bolsa
lista_bolsas = processa_pag_principal(url_pag_principal)

---- Processando artigo  1.. Non-Recyclable Plastics to Pavements..
---- Processando artigo  2.. Techno-Economic Evaluation of Supercritical Water Oxidation ..
---- Processando artigo  3.. Effectiveness of Landfill Liners to Control Transport of PFA..
---- Processando artigo  4.. Repair Strategies for Waste Transfer Station Concrete Overla..
---- Processando artigo  5.. Innovative Technologies to Treat Per- and Polyfluoroalkyl Su..
---- Processando artigo  6.. Polymer-Based Pre-Treatment for Removal of PFAS from Landfil..
---- Processando artigo  7.. The Influence of Social Norms on Recycling Behavior in Urban..
---- Processando artigo  8.. Recycling, Contamination, Markets and MRFs: Practical Strate..
---- Processando artigo  9.. Development of Recognizable Recycled Paper Based Containerbo..
!!! ERRO !!!
---- Processando artigo 10.. Recovering High Value Acids from Anaerobic Co-digestion of M..
---- Processando artigo 11.. Rapid and Cost-Effective Approach to Evaluate the Effectiven..

In [158]:
# Gera o arquivo CSV de saida
df = gera_csv(lista_bolsas, end_arq_csv_saida)

Gerando o arquivo CSV de Saida em ./saida/resultado-raspagem-erefdn.csv.


In [159]:
df

,titulo,instituicao,data_ini,valor,descr_detalhada,links
0,Non-Recyclable Plastics to Pavements,University of Illinois Urbana-Champaign,TBD,"$161,075",This proposal seeks to create high-value and h...,https://erefdn.org/non-recyclable-plastics-to-...
1,Techno-Economic Evaluation of Supercritical Wa...,Duke University,Oct 2020,"$152,000",Landfill leachate and condensate management ca...,https://erefdn.org/techno-economic-evaluation-...
2,Effectiveness of Landfill Liners to Control Tr...,University of Virginia,Jan 2020,"$142,000",The efficacy of Subtitle D landfill lining sys...,https://erefdn.org/effectiveness-of-landfill-l...
3,Repair Strategies for Waste Transfer Station C...,North Carolina State University,Mar 2020,"$195,000","Recently, it has been shown that the premature...",https://erefdn.org/repair-strategies-for-waste...
4,Innovative Technologies to Treat Per- and Poly...,Clarkson University,February 2020,"$145,000",Disposing of leachate-containing per- and poly...,https://erefdn.org/innovative-technologies-to-...
5,Polymer-Based Pre-Treatment for Removal of PFA...,Geosyntec,March 2020,"$105,000",Per- and polyfluoroalkyl substances (PFAS) are...,https://erefdn.org/polymer-based-pre-treatment...
6,The Influence of Social Norms on Recycling Beh...,Stony Brook University,September 2019,"$149,984",The primary goal of this proposal is to test t...,https://erefdn.org/the-influence-of-social-nor...
7,"Recycling, Contamination, Markets and Material...","Skumatz Economic Research Associates, Inc.",October 2019,"$135,000",Material Recovery Facilities (MRFs) can techni...,https://erefdn.org/recycling-contamination-mar...
8,Development of Recognizable Recycled Paper Bas...,North Carolina State University,None,None,There is a growing interest in sustainable pac...,https://erefdn.org/development-of-recognizable...
9,Recovering High Value Acids from Anaerobic Co-...,North Carolina State University and University...,May 2019,"$167,00",Re-engineering the anaerobic co-digestion (co-...,https://erefdn.org/recovering-high-value-acids...


In [160]:
# Teste

In [161]:
soup_pag_princ = obtem_soup(url_pag_principal)

# Obtem a lista de bolsas
lista_bolsas = parse_lista_bolsas(soup_pag_princ)

In [162]:
num_artigo = 6
url_artigo = lista_bolsas[num_artigo]['url_artigo']

# Obtem o objeto soup para uma bolsa especifica
processa_artigo(url_artigo)

{'data_ini': 'September 2019',
 'descr_detalhada': 'The primary goal of this proposal is to test the influence of social norms on recycling\xa0behavior in multifamily buildings in the New York City area. Recycling rates for\xa0multifamily buildings are lower than single family homes, and even when residents are\xa0knowledgeable about what is recyclable, knowledge of non-recyclables lags, leading to\xa0significant contamination. While multifamily recycling research strongly emphasizes the\xa0role of convenience in increasing recycling, once such structural characteristics are in\xa0place there is a need to look to behavioral factors to see further gains. While social norm-based interventions are well-studied in the energy field, there is less work in this area for recycling research, highlighting an important gap.\nThe objectives of the research are to:\nAssess the effectiveness of social-norm based interventions on household recycling behavior in multifamily buildings\nDetermine if sig

In [163]:
# Fim Teste